# Web Scrapping with Beautiful soup

We have to verify the numbers on a web report against our numbers in the database.

the first thing we need is to extract the numbers form the website so we'll use some python scrapping tools.

In [ ]:
import pandas as pd
import numpy as np
import requests

#let's first get the html content of the page containing the table we want
url = '####'
r = requests.get(url)
print(r.content)

In [ ]:
#let's now use html5lib and bs4 to parse the web content
from bs4 import BeautifulSoup
soup = BeautifulSoup(r.content,'html5lib')
print(soup.prettify())

In [ ]:
table = soup.findAll('tr',attrs={'class':['divTableItems','divTableItemsAlt']})
table

In [ ]:
#now that I have the table, I want to create  a dictionary with campus name as key and total as value
total_flash_report = {}
for row in table:
    campus = row.find('td',attrs={'class':'spanCampusCol'})
    campus = campus.a.text
    total = row.find('td',attrs={'class':'spanTotalCol'})
    total = total.text.strip()
    print(campus,total)
    total_flash_report[campus] = total

In [ ]:
total_flash_report

We successfully got the totals for all locations from one url. However, each location name is a link to it's own url where it has more granular data by age group. 

therefore, we need to add some code to retrieve the data from those url's as well and add it to the dictionary

In [ ]:
#let's modify our previous code to retrieve the url for each row, parse that url and 
total_flash_report = {}
for row in table:
    campus_data = {}
    campus = row.find('td',attrs={'class':'spanCampusCol'})
    campus = campus.a.text
    total = row.find('td',attrs={'class':'spanTotalCol'})
    total = total.text.strip()
    campus_url = url + row.a['href']
    campus_content = requests.get(campus_url)
    campus_soup = BeautifulSoup(campus_content.content,'html5lib')
    campus_table = campus_soup.findAll('tr',attrs={'class':['divTableItems','divTableItemsAlt']})
    for row in campus_table:
        group = row.findAll('span',attrs={'style':'float: left;'})
        group = group[0].text
        time = row.findAll('span',attrs={'style':'float: right;'})
        time = time[0].text
        group_time = group + ', ' + time
        value = row.find('td',attrs={'class':'spanTotalCol'})
        value = value.text.strip()
        value
        campus_data[group_time] = value
    total_flash_report[campus] = campus_data

In [ ]:
total_flash_report

## Conclusion

We are ready now to check our scrapped data against any other source :)